<a href="https://colab.research.google.com/github/akaashsidhu/Skin-Cancer-Classification/blob/master/VGG16_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import pickle

import random
from keras.layers import Input
from keras.models import Model
from keras import backend as K
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from typing import Optional
from PIL import Image

Using TensorFlow backend.


In [2]:
search_dir = '/content/drive/My Drive/Data'
os.chdir(search_dir)
os.listdir()

['y_train.pickle', 'X_train.pickle', 'y_test.pickle', 'X_test.pickle']

In [0]:
pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)

In [0]:
X_train = X_train/255
num_classes = len(set(y_train))
y_train = np.eye(num_classes)[np.array(y_train.astype(int)).reshape(-1)]

In [5]:
y_train.shape

(9013, 7)

In [0]:
X_test = X_test/255
num_classes = len(set(y_test))
y_test = np.eye(num_classes)[np.array(y_test.astype(int)).reshape(-1)]

In [0]:
IMG_SIZE = 224

In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [8]:
pre_trained_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")

In [9]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [10]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 512)


In [13]:
from keras import layers
from keras.optimizers import Adam

# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc',f1_m,precision_m, recall_m])


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

In [16]:
batch_size = 64
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size), epochs = epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
141/141 [==============================] - 111s 789ms/step - loss: 1.2524 - acc: 0.6346 - f1_m: 0.5872 - precision_m: 0.6633 - recall_m: 0.5330
Epoch 2/3
141/141 [==============================] - 100s 709ms/step - loss: 1.0706 - acc: 0.6676 - f1_m: 0.6565 - precision_m: 0.7287 - recall_m: 0.5985
Epoch 3/3
141/141 [==============================] - 99s 703ms/step - loss: 1.0137 - acc: 0.6685 - f1_m: 0.6626 - precision_m: 0.7504 - recall_m: 0.5948


In [0]:
for layer in model.layers[:15]:
    layer.trainable = False

for layer in model.layers[15:]:
    layer.trainable = True

In [18]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc',f1_m,precision_m, recall_m])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=3)

In [20]:
batch_size = 64
epochs = 30
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              verbose = 1, callbacks=[learning_rate_reduction])

Epoch 1/30
141/141 [==============================] - 104s 735ms/step - loss: 0.8756 - acc: 0.6941 - f1_m: 0.6935 - precision_m: 0.8111 - recall_m: 0.6088
Epoch 2/30
141/141 [==============================] - 106s 753ms/step - loss: 0.7506 - acc: 0.7292 - f1_m: 0.7239 - precision_m: 0.8281 - recall_m: 0.6445
Epoch 3/30
141/141 [==============================] - 100s 708ms/step - loss: 0.7031 - acc: 0.7471 - f1_m: 0.7388 - precision_m: 0.8371 - recall_m: 0.6627
Epoch 4/30
141/141 [==============================] - 102s 722ms/step - loss: 0.6723 - acc: 0.7566 - f1_m: 0.7493 - precision_m: 0.8384 - recall_m: 0.6789
Epoch 5/30
141/141 [==============================] - 100s 710ms/step - loss: 0.6320 - acc: 0.7733 - f1_m: 0.7659 - precision_m: 0.8481 - recall_m: 0.6996
Epoch 6/30
141/141 [==============================] - 100s 710ms/step - loss: 0.5973 - acc: 0.7852 - f1_m: 0.7802 - precision_m: 0.8540 - recall_m: 0.7193
Epoch 7/30
141/141 [==============================] - 104s 739ms/step 

In [25]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print("Test: accuracy = %f  ;  loss = %f ; f1 score = %f ; precision = %f ; recall = %f " % (accuracy, loss, f1_score, precision, recall))

Test: accuracy = 0.813373  ;  loss = 0.677678 ; f1 score = 0.817414 ; precision = 0.832308 ; recall = 0.803393 


In [0]:
model.save("VGG16.h5")